In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
import scipy
import math
from sklearn import metrics
from sklearn.metrics import * 
from sklearn.preprocessing import label_binarize

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

## Load Data

In [2]:
#Load Labels
escapeLabels  = pd.read_csv("../Data/Original DataFrames/Labels.csv", sep=",", index_col='DEIDNUM').sort_index() #labels for prediction classes 
bestLabels  = pd.read_csv("Preprocessed Data/LabelsBEST.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 
hfactionLabels  = pd.read_csv("Preprocessed Data/LabelsHF-ACTION.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 
guideLabels  = pd.read_csv("Preprocessed Data/LabelsGUIDE-IT.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 
cardShockLabels = pd.read_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/LabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
serialLabels = pd.read_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/LabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()

# Loading scores with mortality labels
escapeHemoScores = pd.read_csv("../Data/Preprocessed Data/ESCAPE_Hemo.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']
cardShockHemoScores = pd.read_csv("../Data Validation/Cardiogenic Shock/Preprocessed Data/CardiogenicShock_Hemo.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']
serialHemoScores = pd.read_csv("../Data Validation/Serial Cardiac Caths/Preprocessed Data/SerialCardiac_Hemo.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']

escapeAllScores = pd.read_csv("../Data/Preprocessed Data/ESCAPE_AllData.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']
hfactionAllScores = pd.read_csv("../Data Validation/HF-ACTION/Preprocessed Data/HF-ACTION_AllData.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']
bestAllScores = pd.read_csv("../Data Validation/BEST/Preprocessed Data/BEST_AllData.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']
guideAllScores = pd.read_csv("../Data Validation/GUIDE-IT/Preprocessed Data/GUIDE-IT_AllData.csv", sep=",", index_col='ID').sort_index()['ScoreDeath']


## Helper Functions

In [3]:
def makeLabels(data, labels):
    lst = []
    idx = sorted(data.index)
    for i in idx:
        lab = labels.loc[i]
        lst.append(lab['Death'])
        
    return pd.DataFrame(lst, columns=['Real'],index=idx)


def convertCARNAEscape(data, scores, missing):
    lst = []
    for r in range(len(data)):
        row = data.iloc[r]
        idx = row.name
        
        if idx in missing:
            lst.append(np.nan)
        else:
            if type(scores.loc[idx]) == pd.Series:
                s = max(scores.loc[idx])
            else:
                s = scores.loc[idx]
            
            #convert score to prob value
            lst.append(CARNAScoreVals(s))

            
    return lst

def convertCARNA(data, scores, missing):
    lst = []
    idx = sorted(set(data.index))
    
    for i in idx:
        if i in missing:
            for row in range(len(data.loc[i])):
                lst.append(np.nan)
        else:
            
            sRows = scores.loc[i]
            
            try:
                sRowLen = len(sRows)
            except:
                sRowLen = 1
            
            if type(data.loc[i]) == pd.Series: #just do once
                
                if type(sRows) == pd.Series:
                    s = max(sRows)
                else:
                    s = sRows
                
                #convert score to prob value
                lst.append(CARNAScoreVals(s))
            
            else:
                for row in range(len(data.loc[i])):
                    if row >= sRowLen:
                        lst.append(np.nan)
                    else:

                        if sRowLen == 1:
                            s = sRows
                        else:
                            s = sRows.iloc[row]

                        #convert score to prob value
                        lst.append(CARNAScoreVals(s))
                                        
    return lst

def CARNAScoreVals(s):
    if s == 5:
        return 0.5
    elif s == 4:
        return 0.35
    elif s == 3:
        return 0.25
    elif s == 2:
        return 0.15
    else:# s == 1:
        return 0.09
       
def convertGWTG(df):
    lstLow = []
    lstHigh = []
    df = df.reset_index()
    idx = sorted(df.index)
    for i in idx:
        val = df.loc[i]['GWTG']
        
        if val == "-":
            val = np.nan
        elif "-" in str(val):
            val = val.split('-')[0]
            
        val = float(val)

        if np.isnan(val):
            lstLow.append(np.nan)
            lstHigh.append(np.nan)
        elif val <= 33:
            lstLow.append(0.01)
            lstHigh.append(np.nan)
        elif val >= 34 and val <= 50:
            lstLow.append(0.01)
            lstHigh.append(0.05)
        elif val >= 51 and val <= 57:
            lstLow.append(0.06)
            lstHigh.append(0.10)
        elif val >= 58 and val <= 61:
            lstLow.append(0.11)
            lstHigh.append(0.15)
        elif val >= 62 and val <= 65:
            lstLow.append(0.16)
            lstHigh.append(0.20)
        elif val >= 66 and val <= 70:
            lstLow.append(0.21)
            lstHigh.append(0.30)
        elif val >= 71 and val <= 74:
            lstLow.append(0.31)
            lstHigh.append(0.4)
        elif val >= 75 and val <= 78:
            lstLow.append(0.41)
            lstHigh.append(0.50)
        else: #val >= 79
            lstLow.append(0.51)
            lstHigh.append(np.nan)
    
    return lstLow, lstHigh

def makeScoreDF(dataset, labels, index, carnaScores=None):
    #Get ESCAPE Score DF
    orig = pd.read_csv("Calculated Scores/ESCAPE/"+ dataset + "_ESCAPE.csv").set_index(index).sort_index()
    lbls = makeLabels(orig, labels)
    escDF = lbls
    escDF['ESCAPE'] = orig[['ESCAPE']]
    
    missing = np.setdiff1d(labels.index, carnaScores.index)
    scrs = convertCARNAEscape(orig, carnaScores, missing)
    escDF['CARNA'] = scrs
    
    
    #Make other scores DF
    #ADHERE
    orig = pd.read_csv("Calculated Scores/ADHERE/"+ dataset + "_ADHERE.csv").set_index(index)
    lbls = makeLabels(orig, labels)
    scrDF = lbls
    
    scrDF[['ADHERE_Low','ADHERE_High']] = orig['ADHERE'].astype(str).str.split('-', expand=True).astype(float)
    scrDF['ADHERE_Low'] = scrDF['ADHERE_Low'] / 100 #split and make btw 0 and 1
    scrDF['ADHERE_High'] = scrDF['ADHERE_High'] / 100

    #GWTG
    try:
        orig = pd.read_csv("Calculated Scores/GWTG/"+ dataset + "_GWTG.csv").set_index(index)
        low, high = convertGWTG(orig)
        scrDF['GWTG_Low'] = low
        scrDF['GWTG_High'] = high
    except:
        scrDF['GWTG_Low'] = np.nan
        scrDF['GWTG_High'] = np.nan
    
    #MAGGIC
    try:
        orig = pd.read_csv("Calculated Scores/MAGGIC/"+ dataset + "_MAGGIC.csv").set_index(index)
        scrDF["MAGGIC Y1"] = orig['Y1'] / 100
        scrDF['MAGGIC Y3'] = orig['Y3'] / 100
    except:
        scrDF["MAGGIC Y1"] = np.nan
        scrDF['MAGGIC Y3'] = np.nan
    
    #Add Optimize and Effect scores
    orig = pd.read_csv("Calculated Scores/OptimizeEffect/"+ dataset + "_optimizeEffectScore.csv").set_index(index)
    scrDF['OPTIMIZE-HF'] = orig['OPTIMIZE-HF']
    scrDF['EFFECT 30 Day'] = orig['EFFECT 30 Day']
    scrDF['EFFECT 1 Year'] = orig['EFFECT 1 Year']
    
    #Add SHFM
    orig = pd.read_csv("Calculated Scores/SHF/"+ dataset + "_SHF.csv").set_index(index).sort_index()
    scrDF["SHFM Y1"] = orig['SHF1'] / 100
    scrDF['SHFM Y3'] = orig['SHF2'] / 100
    scrDF["SHFM Y5"] = orig['SHF5'] / 100
    
    scrs = convertCARNA(orig, carnaScores, missing)
    scrDF['CARNA'] = scrs
    

    return escDF, scrDF



In [4]:
def getAUC(df, scoreList):
    precLst = []
    rocLst = []
    for score in scoreList:
        dfCpy = df[df[score].notna()]
        real = dfCpy['Real']
        scoreVal = dfCpy[score]
        
        if not scoreVal.isnull().all():
            rocAUC = roc_auc_score(real, scoreVal)
            rocLst.append(rocAUC)
        else:
            rocLst.append(np.nan)
    
    return rocLst

In [38]:
# Code from: https://github.com/yandexdataschool/roc_comparison

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
#     print("Z is", z)
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def calc_pvalueV2(aucs, sigma):
    """Computes p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       pvalue
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
#     print("Z is", z)
    return 2 * (1-scipy.stats.norm.cdf(z, loc=0, scale=1))
#     return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)

def compute_ground_truth_statistics(ground_truth):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    return order, label_1_count


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    pvals = calc_pvalueV2(aucs, delongcov)
    return aucs, delongcov, pvals

# Risk Score Comparison
Example steps for one dataset

In [39]:
# Variable defs
datasets = ['ESCAPE','HF-ACTION', 'BEST', 'GUIDE-IT', 'CardShock', 'SerialCardiac']
labels = [escapeLabels, hfactionLabels, bestLabels, guideLabels, cardShockLabels, serialLabels]
index = ['DEIDNUM', 'ID', 'ID', 'ID','ID','ID']
carnaScores = [escapeAllScores, hfactionAllScores, bestAllScores, guideAllScores]
scoreList = ['ADHERE_Low', 'ADHERE_High', 'GWTG_Low', 'GWTG_High','MAGGIC Y1', 'MAGGIC Y3', 'OPTIMIZE-HF', 
             'EFFECT 30 Day','EFFECT 1 Year', 'SHFM Y1', 'SHFM Y3', 'SHFM Y5']

### 1. For each patient record, get the risk score for each scoring method  
    
   If the score does not directly correlate to a probability of the outcome, convert it to the probability value

In [40]:
escapeDF, scoreDF = makeScoreDF(dataset='ESCAPE', labels=escapeLabels, index='DEIDNUM', carnaScores=escapeAllScores)
scoreDF

,Real,ADHERE_Low,ADHERE_High,GWTG_Low,GWTG_High,MAGGIC Y1,MAGGIC Y3,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,SHFM Y1,SHFM Y3,SHFM Y5,CARNA
72,1,0.055,0.132,0.06,0.10,0.316,0.625,0.06,0.122,0.593,0.44,0.69,0.96,0.25
72,1,0.055,0.132,0.06,0.10,0.427,0.756,0.10,0.327,0.593,0.38,0.62,0.93,0.25
81,0,0.055,0.132,0.01,0.05,0.227,0.490,0.06,0.122,0.593,0.31,0.53,0.88,0.25
81,0,0.055,0.132,0.01,0.05,0.122,0.292,0.06,0.122,0.325,0.12,0.23,0.53,0.25
86,0,NaN,NaN,NaN,NaN,NaN,NaN,0.97,0.122,0.325,0.14,0.27,0.58,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99302,1,0.021,0.023,0.01,NaN,0.111,0.269,0.97,0.034,0.129,0.10,0.19,0.45,0.25
99912,0,NaN,NaN,NaN,NaN,NaN,NaN,0.97,0.034,0.129,0.08,0.16,0.39,0.15
99912,0,NaN,NaN,NaN,NaN,NaN,NaN,0.97,0.034,0.129,0.05,0.10,0.27,0.15
99935,0,0.021,0.023,0.01,0.05,0.111,0.269,0.97,0.034,0.325,0.07,0.14,0.35,0.25


### 2. Calculate the AUC curve for each score prediction

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html 

In [41]:
roclst = []
rLst = getAUC(escapeDF, ['ESCAPE'])
roclst.extend(rLst)
    
rLst = getAUC(scoreDF, scoreList)
roclst.extend(rLst)
    
cols = ['ESCAPE', 'ADHERE_Low', 'ADHERE_High', 'GWTG_Low', 'GWTG_High',
           'MAGGIC Y1', 'MAGGIC Y3', 'OPTIMIZE-HF', 'EFFECT 30 Day',
           'EFFECT 1 Year', 'SHFM Y1', 'SHFM Y3', 'SHFM Y5']

rocDF = pd.DataFrame([roclst], columns=cols)
# rocDF = pd.DataFrame(roclst, columns=cols, index=datasets)
rocDF

,ESCAPE,ADHERE_Low,ADHERE_High,GWTG_Low,GWTG_High,MAGGIC Y1,MAGGIC Y3,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,SHFM Y1,SHFM Y3,SHFM Y5
0,0.680515,0.594986,0.594986,0.596784,0.600587,0.639941,0.639941,0.430319,0.55016,0.548222,0.623452,0.622755,0.622373


### 3. Complete Hypothesis Testing


In [42]:
lst = []
#do escape score first
miniDF = escapeDF[['Real', 'CARNA', 'ESCAPE']]
miniDF = miniDF.fillna(0)
aucs, delongcov, pvals = delong_roc_test(ground_truth=miniDF['Real'], predictions_one=miniDF['CARNA'], predictions_two=miniDF['ESCAPE'])
lst.append(abs(pvals[0][0]))

#do other scores
for s in scoreList:        
    miniDF = scoreDF[['Real', 'CARNA', s]]
    miniDF = miniDF.dropna()
#     miniDF = miniDF.fillna(0)

    if len(miniDF.index) == 0: #all NAN
        lst.append(np.nan)
    else:
        aucs, delongcov, pvals = delong_roc_test(ground_truth=miniDF['Real'], predictions_one=miniDF['CARNA'], predictions_two=miniDF[s])
        lst.append(abs(pvals[0][0]))
    
    
cols = ['ESCAPE', 'ADHERE_Low', 'ADHERE_High', 'GWTG_Low', 'GWTG_High','MAGGIC Y1', 'MAGGIC Y3', 'OPTIMIZE-HF', 
             'EFFECT 30 Day','EFFECT 1 Year', 'SHFM Y1', 'SHFM Y3', 'SHFM Y5']

df = pd.DataFrame([lst], columns=cols)
df = df.round(3)
# df.T
df

,ESCAPE,ADHERE_Low,ADHERE_High,GWTG_Low,GWTG_High,MAGGIC Y1,MAGGIC Y3,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,SHFM Y1,SHFM Y3,SHFM Y5
0,0.0,0.256,0.256,0.271,0.23,0.033,0.033,0.005,0.163,0.184,0.0,0.0,0.0


In [32]:
def permutation_test_between_clfs(y_test, pred_proba_1, pred_proba_2, nsamples=1000):
    auc_differences = []
    auc1 = roc_auc_score(y_test.ravel(), pred_proba_1.ravel())
    auc2 = roc_auc_score(y_test.ravel(), pred_proba_2.ravel())
    observed_difference = auc1 - auc2
    for _ in range(nsamples):
        mask = np.random.randint(2, size=len(pred_proba_1.ravel()))
        p1 = np.where(mask, pred_proba_1.ravel(), pred_proba_2.ravel())
        p2 = np.where(mask, pred_proba_2.ravel(), pred_proba_1.ravel())
        auc1 = roc_auc_score(y_test.ravel(), p1)
        auc2 = roc_auc_score(y_test.ravel(), p2)
        auc_differences.append(auc1 - auc2)
    return observed_difference, np.mean(auc_differences >= observed_difference)

In [37]:
# obsDif, meanDif = permutation_test_between_clfs(y_test=miniDF['Real'], pred_proba_1=miniDF['CARNA'], pred_proba_2=miniDF['SHFM Y5'], nsamples=len(miniDF))

# print("Observed Diff:", obsDif, "p(diff>=)", meanDif)

In [36]:
lst = []
#do escape score first
miniDF = escapeDF[['Real', 'CARNA', 'ESCAPE']]
miniDF = miniDF.fillna(0)
obsDif, meanDif = permutation_test_between_clfs(y_test=miniDF['Real'], pred_proba_1=miniDF['CARNA'], pred_proba_2=miniDF['ESCAPE'], nsamples=len(miniDF))
lst.append(meanDif)

#do other scores
for s in scoreList:        
    miniDF = scoreDF[['Real', 'CARNA', s]]
    miniDF = miniDF.dropna()
#     miniDF = miniDF.fillna(0)

    if len(miniDF.index) == 0: #all NAN
        lst.append(np.nan)
    else:
        obsDif, meanDif = permutation_test_between_clfs(y_test=miniDF['Real'], pred_proba_1=miniDF['CARNA'], pred_proba_2=miniDF[s], nsamples=len(miniDF))
        lst.append(meanDif)
    
    
cols = ['ESCAPE', 'ADHERE_Low', 'ADHERE_High', 'GWTG_Low', 'GWTG_High','MAGGIC Y1', 'MAGGIC Y3', 'OPTIMIZE-HF', 
             'EFFECT 30 Day','EFFECT 1 Year', 'SHFM Y1', 'SHFM Y3', 'SHFM Y5']

df = pd.DataFrame([lst], columns=cols)
df = df.round(3)
# df.T
df

,ESCAPE,ADHERE_Low,ADHERE_High,GWTG_Low,GWTG_High,MAGGIC Y1,MAGGIC Y3,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,SHFM Y1,SHFM Y3,SHFM Y5
0,1.0,0.744,0.749,0.777,0.754,0.975,0.956,0.005,0.893,0.881,1.0,1.0,0.997
